# Обработка данных

**Разделы:**
- [Импортируем библиотеки и загрузим данные](#импортируем-библиотеки-и-загрузим-данные)
- [Анализ и очистка данных](#анализ-и-очистка-данных)
- [Feature Enginnering](#Feature-Enginnering)
    - [Первичная-обработка](#первичная-обработка)
    - [Кодирование категориальных признаков](#кодирование-категориальных-признаков)
    - [Удаление ненужных столбцов](#удаление-ненужных-столбцов)
- [Разделение и сохранение данных](#разделение-и-сохранение-данных)

## Импортируем библиотеки и загрузим данные

In [1]:
import pandas as pd
import numpy as np
import os
import sys

from sklearn.preprocessing import StandardScaler, RobustScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split

In [2]:
# Загрузка данных из data/raw/
data_path = os.path.join('..', 'data', 'raw', 'diabetes_dataset.csv')
df = pd.read_csv(data_path)
df

,age,gender,ethnicity,education_level,income_level,employment_status,smoking_status,alcohol_consumption_per_week,physical_activity_minutes_per_week,diet_score,...,hdl_cholesterol,ldl_cholesterol,triglycerides,glucose_fasting,glucose_postprandial,insulin_level,hba1c,diabetes_risk_score,diabetes_stage,diagnosed_diabetes
0,58,Male,Asian,Highschool,Lower-Middle,Employed,Never,0,215,5.7,...,41,160,145,136,236,6.36,8.18,29.6,Type 2,1
1,48,Female,White,Highschool,Middle,Employed,Former,1,143,6.7,...,55,50,30,93,150,2.00,5.63,23.0,No Diabetes,0
2,60,Male,Hispanic,Highschool,Middle,Unemployed,Never,1,57,6.4,...,66,99,36,118,195,5.07,7.51,44.7,Type 2,1
3,74,Female,Black,Highschool,Low,Retired,Never,0,49,3.4,...,50,79,140,139,253,5.28,9.03,38.2,Type 2,1
4,46,Male,White,Graduate,Middle,Retired,Never,1,109,7.2,...,52,125,160,137,184,12.74,7.20,23.5,Type 2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,46,Male,Other,Graduate,Upper-Middle,Unemployed,Former,1,136,8.3,...,45,150,116,113,109,14.58,5.55,26.0,Pre-Diabetes,0
99996,41,Female,White,Graduate,Middle,Employed,Never,3,76,8.8,...,55,123,146,96,146,9.02,5.97,24.4,Pre-Diabetes,0
99997,57,Female,Black,No formal,Upper-Middle,Employed,Former,4,121,9.9,...,50,111,184,93,132,2.57,5.21,27.6,No Diabetes,0
99998,47,Female,Black,Highschool,Lower-Middle,Retired,Never,3,52,5.9,...,68,91,116,106,117,9.81,5.53,26.4,Pre-Diabetes,0


In [3]:
TARGET = 'diagnosed_diabetes'

## Анализ и очистка данных

In [4]:
print("Проверка пропусков:")
print(df.isnull().sum())

Проверка пропусков:
age                                   0
gender                                0
ethnicity                             0
education_level                       0
income_level                          0
employment_status                     0
smoking_status                        0
alcohol_consumption_per_week          0
physical_activity_minutes_per_week    0
diet_score                            0
sleep_hours_per_day                   0
screen_time_hours_per_day             0
family_history_diabetes               0
hypertension_history                  0
cardiovascular_history                0
bmi                                   0
waist_to_hip_ratio                    0
systolic_bp                           0
diastolic_bp                          0
heart_rate                            0
cholesterol_total                     0
hdl_cholesterol                       0
ldl_cholesterol                       0
triglycerides                         0
glucose_fasting     

Пропусков нет, очень хорошо

In [5]:
numeric_features = df.select_dtypes(include=[np.number]).columns.tolist()

# Или если нужно исключить целевую переменную
numeric_features_without_target = [col for col in numeric_features if col != 'diagnosed_diabetes']
print(f"\n📊 Числовые признаки (без целевой): {len(numeric_features_without_target)}")
print(df[numeric_features_without_target].describe())


📊 Числовые признаки (без целевой): 23
                age  alcohol_consumption_per_week  \
count  100000.00000                 100000.000000   
mean       50.12041                      2.003670   
std        15.60460                      1.417779   
min        18.00000                      0.000000   
25%        39.00000                      1.000000   
50%        50.00000                      2.000000   
75%        61.00000                      3.000000   
max        90.00000                     10.000000   

       physical_activity_minutes_per_week     diet_score  sleep_hours_per_day  \
count                       100000.000000  100000.000000        100000.000000   
mean                           118.911640       5.994787             6.997818   
std                             84.409662       1.780954             1.094622   
min                              0.000000       0.000000             3.000000   
25%                             57.000000       4.800000             6.300000 

## Feature Enginnering

Что мы будем делать:

1. Первичная обработка
     * `'age_category'`: 
          * Создание возрастных групп 
               * young - молодой
               * adult - взрослый
               * middle_aged - средний возраст
               * elderly - пожилой
     * `'bmi_category'` 
          * Медицинские категории BMI
               * underweight - недостаточный вес
               * normal - нормальный вес
               * overweight - избыточный вес
               * obese - ожирение
     * `'glucose_risk'`
          * Комбинированный риск по глюкозе (норма, преддиабет, диабет) + HbA1c
     * `'lifestyle_score'` 
          * Композитный показатель образа жизни (диета + активность + сон - алкоголь - курение)
2. Кодирование категориальных признаков
     * `OneHotEncoder`: [`'gender'`, `'ethnicity'`, `'employment_status'`, `'diabetes_stage'`]
          * Признаки без естественной иерархии
          * Сохраняет информацию о всех категориях
     * 'OrdinalEncoder':
          * `'education_level'`: Primary(0) < Secondary(1) < Undergraduate(2) < Postgraduate(3)
          * `'income_level'`: Low(0) < Medium(1) < High(2)
          * `'smoking_status'`: Never(0) < Former(1) < Current(2)
          * Сохраняет естественный порядок и логичность изменения
3. Масштабирование числовых признаков
   * `StandardScaler`: [`'age'`, `'bmi'`, `'cholesterol_total'`, `'blood_pressure'`, `'heart_rate'`]
        * Признаки с нормальным распределением
        * Стандартизация для линейных моделей
   * `RobustScaler`: [`'glucose_fasting'`, `'glucose_postprandial'`, `'hba1c'`, `'insulin_level'`]
        * Медицинские показатели с возможными выбросами
        * Устойчив к аномалиям


In [6]:
# создадим копию данных для преобразований
df_processed = df.copy()

### Первичная обработка

In [7]:
df_processed['age_category'] = pd.cut(df_processed['age'], 
                           bins=[0, 35, 50, 65, 100],
                           labels=['young', 'adult', 'middle_aged', 'elderly'])

Сделаем показатель BMI категориальным 

![BMI_расшифровка](..\data\images\eda\bmi_values.png)


In [8]:
df_processed['bmi_category'] = pd.cut(df_processed['bmi'],
                           bins=[0, 18.5, 25, 30, 100],
                           labels=['underweight', 'normal', 'overweight', 'obese'])

Обоснованость создания параметра риска по глюкозе вы можете прочитать в этой [статье](https://kdl.ru/patient/blog/diabet-pod-kontrolem)

In [9]:
def get_glucose_risk(row):
    if row['glucose_fasting'] < 100 and row['hba1c'] < 5.7:
        return 'normal'
    elif row['glucose_fasting'] < 126 and row['hba1c'] < 6.5:
        return 'prediabetes'
    else:
        return 'diabetes'
    
df_processed['glucose_risk'] = df_processed.apply(get_glucose_risk, axis=1)

Создадим так-же составной показатель для представления уровня здорового образа жизни человека.
(эксперементальный параметр)

In [10]:
df_processed['lifestyle_score'] = (
    df_processed['diet_score'] + 
    (df_processed['physical_activity_minutes_per_week'] / 100) +
    df_processed['sleep_hours_per_day'] -
    (df_processed['alcohol_consumption_per_week'] * 2) -
    (df_processed['screen_time_hours_per_day'] * 0.5)
)

### Кодирование категориальных признаков

Используем `OneHotEncoder`

In [11]:
onehot_columns = ['gender', 'ethnicity', 'employment_status', 'diabetes_stage']
print(f"OneHot кодирование для: {onehot_columns}")

onehot_encoder = OneHotEncoder(sparse_output=False, drop='first')
onehot_encoded = onehot_encoder.fit_transform(df_processed[onehot_columns])

# создаем DataFrame с one-hot признаками
onehot_feature_names = onehot_encoder.get_feature_names_out(onehot_columns)
df_onehot = pd.DataFrame(onehot_encoded, columns=onehot_feature_names, index=df_processed.index)

# удаляем исходные колонки и добавляем one-hot
df_processed = df_processed.drop(onehot_columns, axis=1)
df_processed = pd.concat([df_processed, df_onehot], axis=1)

print(f"Добавлено {len(onehot_feature_names)} новых признаков")
print(f"Размер новых данных: {df_processed.shape}")

OneHot кодирование для: ['gender', 'ethnicity', 'employment_status', 'diabetes_stage']
Добавлено 13 новых признаков
Размер новых данных: (100000, 44)


Используем `OrdinalEncoder`

In [12]:
df_processed['education_level'].unique()

array(['Highschool', 'Graduate', 'Postgraduate', 'No formal'],
      dtype=object)

In [13]:
ordinal_columns = ['education_level', 'income_level', 'smoking_status']
education_order = ['No formal', 'Highschool', 'Graduate', 'Postgraduate']
income_order = ['Lower-Middle', 'Middle', 'Low', 'Upper-Middle', 'High']
smoking_order = ['Never', 'Former', 'Current']

print(f"Ordinal кодирование для: {ordinal_columns}")

ordinal_encoder_edu = OrdinalEncoder(categories=[education_order])
df_processed['education_level_encoded'] = ordinal_encoder_edu.fit_transform(df_processed[['education_level']])

ordinal_encoder_inc = OrdinalEncoder(categories=[income_order])
df_processed['income_level_encoded'] = ordinal_encoder_inc.fit_transform(df_processed[['income_level']])

ordinal_encoder_smk = OrdinalEncoder(categories=[smoking_order])
df_processed['smoking_status_encoded'] = ordinal_encoder_smk.fit_transform(df_processed[['smoking_status']])
print(f"Размер новых данных: {df_processed.shape}")

Ordinal кодирование для: ['education_level', 'income_level', 'smoking_status']
Размер новых данных: (100000, 47)


### Масштабирование числовых признаков

StandardScaler

In [14]:
standard_columns = ['age', 'bmi', 'cholesterol_total', 'heart_rate']
print(f"StandardScaler для: {standard_columns}")

standard_scaler = StandardScaler()
df_processed[standard_columns] = standard_scaler.fit_transform(df_processed[standard_columns])

print(f"\nМасштабирование завершено. Диапазоны:")
for col in standard_columns:
    print(f"  {col:20}: {df_processed[col].min():.1f} - {df_processed[col].max():.1f}")

StandardScaler для: ['age', 'bmi', 'cholesterol_total', 'heart_rate']

Масштабирование завершено. Диапазоны:
  age                 : -2.1 - 2.6
  bmi                 : -3.0 - 3.8
  cholesterol_total   : -2.7 - 4.1
  heart_rate          : -3.5 - 4.2


RobustScaler

In [15]:
robust_columns = ['glucose_fasting', 'glucose_postprandial', 'hba1c', 'insulin_level']
print(f"RobustScaler для: {robust_columns}")

robust_scaler = RobustScaler()
df_processed[robust_columns] = robust_scaler.fit_transform(df_processed[robust_columns])

print(f"\nМасштабирование завершено. Диапазоны:")
for col in robust_columns:
    print(f"  {col:20}: {df_processed[col].min():.1f} - {df_processed[col].max():.1f}")

RobustScaler для: ['glucose_fasting', 'glucose_postprandial', 'hba1c', 'insulin_level']

Масштабирование завершено. Диапазоны:
  glucose_fasting     : -2.8 - 3.4
  glucose_postprandial: -2.1 - 3.0
  hba1c               : -2.3 - 3.0
  insulin_level       : -0.9 - 3.2


### Удаление ненужных столбцов и выбросов

Сначало посмотрим какие у нас есть столбцы на данный момент:

In [16]:
columns = df_processed.columns.tolist()

for i in range(0, len(columns), 3):
    # форматируем вывод для лучшей читаемости
    row_columns = columns[i:i+3]
    formatted_columns = []
    
    for col in row_columns:
        # Обрезаем длинные названия для удобства чтения
        if len(col) > 25:
            col_display = col[:20] + "..."
        else:
            col_display = col
        formatted_columns.append(f"{col_display:<25}")
    
    print("  ".join(formatted_columns))

age                        education_level            income_level             
smoking_status             alcohol_consumption_...    physical_activity_mi...  
diet_score                 sleep_hours_per_day        screen_time_hours_per_day
family_history_diabetes    hypertension_history       cardiovascular_history   
bmi                        waist_to_hip_ratio         systolic_bp              
diastolic_bp               heart_rate                 cholesterol_total        
hdl_cholesterol            ldl_cholesterol            triglycerides            
glucose_fasting            glucose_postprandial       insulin_level            
hba1c                      diabetes_risk_score        diagnosed_diabetes       
age_category               bmi_category               glucose_risk             
lifestyle_score            gender_Male                gender_Other             
ethnicity_Black            ethnicity_Hispanic         ethnicity_Other          
ethnicity_White            employment_st

Удалим исходные не нужные столбцы 

In [17]:
print(f"Кол. фич перед удалением: {len(df_processed.columns)}")

columns_to_drop = [
    'education_level', 'income_level', 'smoking_status',  # исходные категориальные
    'age_category', 'bmi_category', 'glucose_risk',       # созданные нами категории
    'diabetes_risk_score'                                 # агрегированный показатель
]

df_processed = df_processed.drop(columns_to_drop, axis=1)

print(f"Кол. фич после удаленгия: {len(df_processed.columns)}")

Кол. фич перед удалением: 47
Кол. фич после удаленгия: 40


## Разделение и сохранение данных

In [18]:
X = df_processed.drop([TARGET], axis=1)
y = df_processed[TARGET]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Итог раздения:\n")
print(f"   X_train: {X_train.shape}")
print(f"   X_test:  {X_test.shape}\n")
print(f"   y_train: {y_train.shape}")
print(f"   y_test:  {y_test.shape}")

Итог раздения:

   X_train: (80000, 39)
   X_test:  (20000, 39)

   y_train: (80000,)
   y_test:  (20000,)


In [19]:
os.makedirs('../data/processed', exist_ok=True)

X_train.to_csv('../data/processed/X_train.csv', index=False)
X_test.to_csv('../data/processed/X_test.csv', index=False)
y_train.to_csv('../data/processed/y_train.csv', index=False)
y_test.to_csv('../data/processed/y_test.csv', index=False)